In [115]:
from random import normalvariate, randint
from itertools import count
from datetime import datetime
from itertools import groupby

from scipy.stats import normaltest
from itertools import filterfalse
from itertools import islice

In [1]:
def read_data(filename):
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(',')
            timestamp, value = map(int, data)
            yield datetime.fromtimestamp(timestamp), value



def read_fake_data(filename):
    for timestamp in count():
        # insert anomalous data poit aooroxinately once a week
        if randint(0, 7 * 60 * 60 * 24 - 1) ==1:
            value = normalvariate(0,1)

        else:
            value = 100
        
        yield datetime.fromtimestamp(timestamp), value

def groupby_day(iterable):
    key = lambda row: row[0].day
    for data, data_group in groupby(iterable, key):
        yield list(data_group)

def is_normal(data, threshold=1e-3):
    _, values = zip(*data)
    k2, p_values = normaltest(values)
    if p_values<threshold:
        return False
    return True

def filter_anomalous_groups(data):
    """Filter down the dataset only to inputs that don't pass the test"""

    yield from filterfalse(is_normal, data)


def filter_anomalous_data(data):
    data_group= groupby_day(data)
    yield from filter_anomalous_groups(data_group)